In [1]:
import pandas as pd
df = pd.read_csv("datascience.csv", encoding='gb18030')
df.head()

,title,author,content
0,大数据产业迎政策暖风 最新大数据概念股一览,财经热点扒客,大数据产业发展受到国家重视，而大数据已经上升为国家战略，未来发展前景很广阔。大数据产业“十三...
1,Google发布机器学习平台Tensorflow游乐场～带你一起玩神经网络！,硅谷周边,点击上方“硅谷周边”关注我，收到最新的文章哦！昨天，Google发布了Tensorflow游...
2,李克强：中国大数据和云计算产业是开放的,苏州高新区金融办,国务院总理李克强当地时间20日上午在纽约下榻饭店同美国经济、金融、智库、媒体等各界人士座谈，...
3,全峰集团持续挖掘大数据,快递物流网,2016年，全峰集团持续挖掘大数据、云计算、“互联网+”等前沿技术和物流快递的融合，并通过优...
4,第366期【微理工】贵州理工学院召开大数据分析与应用专题分享会,贵州理工学院,贵州理工学院召开大数据分析与应用专题分享会 借“创响中国”贵安站巡回接力活动暨2016贵安大...


In [2]:
df.shape

(1024, 3)

In [4]:
import jieba

def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [5]:
df["content_cutted"] = df.content.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 2.651 seconds.
Prefix dict has been built succesfully.


In [6]:
df.content_cutted.head()

0    大 数据 产业 发展 受到 国家 重视 ， 而 大 数据 已经 上升 为 国家 战略 ， 未...
1    点击 上方 “ 硅谷 周边 ” 关注 我 ， 收到 最新 的 文章 哦 ！ 昨天 ， Goo...
2    国务院 总理 李克强 当地 时间 20 日 上午 在 纽约 下榻 饭店 同 美国 经济 、 ...
3    2016 年 ， 全峰 集团 持续 挖掘 大 数据 、 云 计算 、 “ 互联网 + ” 等...
4    贵州 理工学院 召开 大 数据分析 与 应用 专题 分享 会   借 “ 创响 中国 ” 贵...
Name: content_cutted, dtype: object

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.content_cutted)


In [8]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 5
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [9]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_jobs=1, n_topics=5, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [10]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [11]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
学习 模型 使用 算法 方法 机器 可视化 神经网络 特征 处理 计算 系统 不同 数据库 训练 分类 基于 工具 一种 深度
Topic #1:
这个 就是 可能 如果 他们 没有 自己 很多 什么 不是 但是 这样 因为 一些 时候 现在 用户 所以 非常 已经
Topic #2:
企业 平台 服务 管理 互联网 公司 行业 数据分析 业务 用户 产品 金融 创新 客户 实现 系统 能力 产业 工作 价值
Topic #3:
中国 2016 电子 增长 10 市场 城市 2015 关注 人口 检索 30 或者 其中 阅读 应当 美国 全国 同比 20
Topic #4:
人工智能 学习 领域 智能 机器人 机器 人类 公司 深度 研究 未来 识别 已经 医疗 系统 计算机 目前 语音 百度 方面
()


In [12]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      25.400115        1       1  0.086035 -0.123999
0      24.758016        1       2 -0.147904  0.040901
1      23.997452        1       3 -0.098711  0.087754
4      14.362051        1       4 -0.023022 -0.126969
3      11.482366        1       5  0.183602  0.122313, topic_info=     Category         Freq  Term        Total  loglift  logprob
term                                                           
122   Default  2776.000000  人工智能  2776.000000  30.0000  30.0000
364   Default  4212.000000    学习  4212.000000  29.0000  29.0000
148   Default  3222.000000    企业  3222.000000  28.0000  28.0000
586   Default  1452.000000    智能  1452.000000  27.0000  27.0000
616   Default  1111.000000   机器人  1111.000000  26.0000  26.0000
615   Default  2189.000000    机器  2189.000000  25.0000  25.0000
992   Default  2006.000000    领域  2006.000000  24.0000  24.0000
91    Default  1769.000000    中国  1769.000000  23.0000  23.0000
125   Default  1018.000000    人类  1018.000000  22.0000  22.0000
779   Default  1851.000000    算法  1851.000000  21.0000  21.0000
645   Default  1778.000000    模型  1778.000000  20.0000  20.0000
929   Default  2254.000000    这个  2254.000000  19.0000  19.0000
719   Default   717.000000    电子   717.000000  18.0000  18.0000
18    Default  1001.000000  2016  1001.000000  17.0000  17.0000
193   Default  2440.000000    公司  2440.000000  16.0000  16.0000
401   Default  1790.000000    就是  1790.000000  15.0000  15.0000
780   Default  1363.000000    管理  1363.000000  14.0000  14.0000
678   Default  1342.000000    深度  1342.000000  13.0000  13.0000
607   Default  1749.000000    服务  1749.000000  12.0000  12.0000
424   Default  1736.000000    平台  1736.000000  11.0000  11.0000
340   Default   812.000000    增长   812.000000  10.0000  10.0000
360   Default  1658.000000    如果  1658.000000   9.0000   9.0000
331   Default   797.000000    城市   797.000000   8.0000   8.0000
1     Default   978.000000    10   978.000000   7.0000   7.0000
136   Default  1330.000000    他们  1330.000000   6.0000   6.0000
417   Default  1409.000000    市场  1409.000000   5.0000   5.0000
129   Default  1060.000000    什么  1060.000000   4.0000   4.0000
289   Default  1018.000000   可视化  1018.000000   3.0000   3.0000
454   Default  1232.000000    很多  1232.000000   2.0000   2.0000
109   Default  1716.000000   互联网  1716.000000   1.0000   1.0000
...       ...          ...   ...          ...      ...      ...
7      Topic5   200.436025    14   261.115114   1.8999  -5.4791
575    Topic5   177.930823    旅游   235.353895   1.8847  -5.5982
5      Topic5   270.116612    12   363.303328   1.8680  -5.1808
347    Topic5   263.032178    大众   355.380966   1.8634  -5.2074
326    Topic5   225.087150    地区   312.488312   1.8363  -5.3631
8      Topic5   227.960512    15   316.502218   1.8362  -5.3505
18     Topic5   699.057159  2016  1001.574046   1.8048  -4.2299
340    Topic5   568.303563    增长   812.642566   1.8067  -4.4370
187    Topic5   275.833830    全国   389.799400   1.8185  -5.1598
4      Topic5   252.945382    11   353.324925   1.8301  -5.2465
17     Topic5   439.166596  2015   707.294759   1.6878  -4.6948
1      Topic5   556.132409    10   978.589287   1.5993  -4.4586
331    Topic5   471.803131    城市   797.034786   1.6400  -4.6231
91     Topic5   864.268010    中国  1769.363007   1.4479  -4.0177
22     Topic5   299.927905    30   447.424045   1.7644  -5.0761
417    Topic5   505.959206    市场  1409.166017   1.1401  -4.5532
228    Topic5   232.676456    分别   349.331746   1.7580  -5.3300
969    Topic5   286.695037    阅读   561.833328   1.4916  -5.1212
198    Topic5   335.463084    关注   929.880255   1.1448  -4.9641
13     Topic5   272.143550    20   545.659028   1.4687  -5.1733
812    Topic5   283.209140    美国   649.534326   1.3343  -5.1334
203    Topic5   293.144837    其中   825.759961   1.1287  -5.0990
484    Topic5   293.957990    或者  1125.908242   0.8214  -5.

In [13]:
n_topics = 10
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)
print_top_words(lda, tf_feature_names, n_top_words)
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

Topic #0:
这个 就是 如果 可能 用户 一些 什么 很多 没有 这样 时候 但是 因为 不是 所以 不同 如何 使用 或者 非常
Topic #1:
中国 孩子 增长 市场 2016 学生 10 2015 城市 自己 人口 大众 关注 其中 教育 同比 没有 美国 投资 这个
Topic #2:
data 变量 距离 http 样本 com www 检验 方法 分布 计算 聚类 如下 分类 之间 两个 一种 差异 表示 序列
Topic #3:
电子 采集 应当 或者 案件 保护 规定 信用卡 收集 是否 提取 设备 法律 申请 法院 系统 记录 相关 要求 无法
Topic #4:
系统 检索 交通 平台 专利 智能 监控 采集 海量 管理 搜索 智慧 出行 视频 车辆 计算 实现 基于 数据库 存储
Topic #5:
可视化 使用 工具 数据库 存储 hadoop 处理 图表 数据仓库 支持 查询 开发 设计 sql 开源 用于 创建 用户 基于 软件
Topic #6:
学习 算法 模型 机器 深度 神经网络 方法 训练 特征 分类 网络 使用 基于 介绍 研究 预测 回归 函数 参数 图片
Topic #7:
企业 管理 服务 互联网 金融 客户 行业 平台 实现 建立 社会 政府 研究 资源 安全 时代 利用 传统 价值 医疗
Topic #8:
人工智能 领域 机器人 智能 公司 人类 机器 学习 未来 已经 研究 他们 识别 可能 计算机 目前 语音 工作 现在 能够
Topic #9:
用户 公司 企业 互联网 平台 中国 数据分析 行业 产业 产品 创新 项目 2016 服务 工作 科技 相关 业务 移动 市场
()


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      21.197996        1       1 -0.049014  0.069388
7      15.385974        1       2  0.152176 -0.017778
8      13.667776        1       3  0.104821  0.119949
9      13.138773        1       4  0.154289 -0.004241
6      10.446711        1       5 -0.103358  0.084971
1      10.393182        1       6  0.061392  0.097447
5       8.147710        1       7 -0.002921 -0.004703
3       3.116697        1       8 -0.088856 -0.302935
4       2.908321        1       9  0.104191 -0.100593
2       1.596859        1      10 -0.332721  0.058496, topic_info=     Category         Freq  Term        Total  loglift  logprob
term                                                           
364   Default  4279.000000    学习  4279.000000  30.0000  30.0000
122   Default  2761.000000  人工智能  2761.000000  29.0000  29.0000
785   Default  2011.000000    系统  2011.000000  28.0000  28.0000
148   Default  3179.000000    企业  3179.000000  27.0000  27.0000
615   Default  2214.000000    机器  2214.000000  26.0000  26.0000
779   Default  1891.000000    算法  1891.000000  25.0000  25.0000
645   Default  1817.000000    模型  1817.000000  24.0000  24.0000
573   Default  1560.000000    方法  1560.000000  23.0000  23.0000
719   Default   720.000000    电子   720.000000  22.0000  22.0000
586   Default  1450.000000    智能  1450.000000  21.0000  21.0000
424   Default  1723.000000    平台  1723.000000  20.0000  20.0000
289   Default  1022.000000   可视化  1022.000000  19.0000  19.0000
616   Default  1105.000000   机器人  1105.000000  18.0000  18.0000
756   Default   935.000000  神经网络   935.000000  17.0000  17.0000
32    Default   533.000000  data   533.000000  16.0000  16.0000
166   Default  2248.000000    使用  2248.000000  15.0000  15.0000
869   Default  1448.000000    计算  1448.000000  14.0000  14.0000
484   Default  1140.000000    或者  1140.000000  13.0000  13.0000
956   Default   755.000000    采集   755.000000  12.0000  12.0000
678   Default  1360.000000    深度  1360.000000  11.0000  11.0000
780   Default  1353.000000    管理  1353.000000  10.0000  10.0000
91    Default  1729.000000    中国  1729.000000   9.0000   9.0000
631   Default   448.000000    样本   448.000000   8.0000   8.0000
125   Default  1013.000000    人类  1013.000000   7.0000   7.0000
232   Default   689.000000    分类   689.000000   6.0000   6.0000
31    Default   514.000000   com   514.000000   5.0000   5.0000
38    Default   410.000000  http   410.000000   4.0000   4.0000
714   Default  2492.000000    用户  2492.000000   3.0000   3.0000
193   Default  2412.000000    公司  2412.000000   2.0000   2.0000
281   Default   357.000000    变量   357.000000   1.0000   1.0000
...       ...          ...   ...          ...      ...      ...
357   Topic10   109.366687    如下   314.276863   3.0816  -4.1122
996   Topic10    43.792629    驾驶   175.114397   2.7512  -5.0274
696   Topic10    50.493897    独立   225.191733   2.6420  -4.8850
69    Topic10    63.834082    下面   317.127989   2.5341  -4.6506
745   Topic10    28.918897    矩阵   157.003159   2.4454  -5.4424
299   Topic10    57.545836    向量   312.986407   2.4436  -4.7543
975   Topic10    34.566173    随机   194.109806   2.4116  -5.2640
252   Topic10    24.186988    医学   151.186380   2.3044  -5.6210
200   Topic10    63.666913    关联   403.911693   2.2896  -4.6532
87    Topic10    81.282662    两个   537.712297   2.2477  -4.4089
232   Topic10    98.462864    分类   689.113226   2.1914  -4.2172
88    Topic10    22.428606    两种   163.299289   2.1519  -5.6965
375   Topic10    44.013163    定义   331.583626   2.1177  -5.0224
734   Topic10    19.514563    相同   147.646506   2.1135  -5.8357
290   Topic10    30.787730    各个   233.946406   2.1092  -5.3797
43    Topic10    21.466857    qq   163.564069   2.1064  -5.7403
425   Topic10    38.263983    平均   307.137710   2.0543  -5.1623
168   Topic10    41.408162    例子   338.056226   2.0374  -5.0834
102   Topic10    85.844681    之间   705.819595